In [1]:
from data_loader import load_data

In [2]:
import numpy as np
import random

class GeneticAlgorithm:
    def __init__(self, num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix,
                 population_size=100, generations=500, mutation_rate=0.1, crossover_rate=0.8):
        self.num_depots = num_depots
        self.num_customers = num_customers
        self.depot_capacities = np.array(depot_capacities)
        self.setup_costs = np.array(setup_costs)
        self.customer_demands = np.array(customer_demands)
        self.cost_matrix = np.array(cost_matrix)
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

    def initialize_population(self):
        # Each individual is a list of integers, where index is the customer and value is the depot assignment
        return [np.random.randint(0, self.num_depots, size=self.num_customers) for _ in range(self.population_size)]

    def fitness(self, individual):
        # Calculate the total cost for a given depot-customer assignment
        total_cost = 0
        depot_loads = np.zeros(self.num_depots)

        for customer, depot in enumerate(individual):
            depot_loads[depot] += self.customer_demands[customer]
            total_cost += self.cost_matrix[customer][depot]

        # Add depot setup costs for depots that are used
        used_depots = depot_loads > 0
        total_cost += np.sum(self.setup_costs[used_depots])

        # Penalize solutions that exceed depot capacities
        capacity_excess = np.maximum(0, depot_loads - self.depot_capacities)
        penalty = np.sum(capacity_excess) * 1e4  # Ceza katsayısını 1e4'e düşürdük
        total_cost += penalty

        return total_cost

    def selection(self, population, fitness_scores):
        # Turnuva Seçimi
        tournament_size = 5
        selected = random.sample(list(zip(population, fitness_scores)), tournament_size)
        selected = sorted(selected, key=lambda x: x[1])  # Fitness'e göre sırala
        return selected[0][0], selected[1][0]
    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            # Single-point crossover
            point = random.randint(1, self.num_customers - 1)
            child1 = np.concatenate((parent1[:point], parent2[point:]))
            child2 = np.concatenate((parent2[:point], parent1[point:]))
            return child1, child2
        return parent1.copy(), parent2.copy()

    def mutate(self, individual):
        if random.random() < self.mutation_rate:
            # Swap mutasyonu: İki müşterinin depot atamasını değiştir
            customer1, customer2 = random.sample(range(self.num_customers), 2)
            individual[customer1], individual[customer2] = individual[customer2], individual[customer1]
        return individual

    def evolve(self):
        population = self.initialize_population()
        best_solution = None
        best_cost = float('inf')

        for generation in range(self.generations):
            fitness_scores = [self.fitness(individual) for individual in population]
            new_population = []

            for _ in range(self.population_size // 2):
                parent1, parent2 = self.selection(population, fitness_scores)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                new_population.extend([child1, child2])

            population = new_population

            # Track the best solution
            min_cost = min(fitness_scores)
            if min_cost < best_cost:
                best_cost = min_cost
                best_solution = population[np.argmin(fitness_scores)]

            print(f"Generation {generation + 1}, Best Cost: {best_cost}")
        early_stop_generations = 10  # Son 50 nesilde iyileşme olmazsa durdur
        no_improvement_count = 0
        for generation in range(self.generations):
            fitness_scores = [self.fitness(individual) for individual in population]
            min_cost = min(fitness_scores)
            
            if min_cost < best_cost:
                best_cost = min_cost
                best_solution = population[np.argmin(fitness_scores)]
                no_improvement_count = 0
            else:
                no_improvement_count += 1
            
            if no_improvement_count >= early_stop_generations:
                print(f"Erken durdurma: İyileşme görülmedi. {generation + 1}. nesilde durduruldu.")
                break
        return best_solution, best_cost

In [5]:
# Load the problem data
file_path = "Dataset/wl_25"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1000,  # Adjust as needed
    generations=1500,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 1640840.8375000001
Generation 2, Best Cost: 1611810.4124999999
Generation 3, Best Cost: 1483793.325
Generation 4, Best Cost: 1483793.325
Generation 5, Best Cost: 1468480.825
Generation 6, Best Cost: 1412926.3124999998
Generation 7, Best Cost: 1357814.1625000003
Generation 8, Best Cost: 1283834.9375000005
Generation 9, Best Cost: 1270908.4749999999
Generation 10, Best Cost: 1231394.7625
Generation 11, Best Cost: 1172617.5625
Generation 12, Best Cost: 1140344.9125
Generation 13, Best Cost: 1121241.4000000001
Generation 14, Best Cost: 1100687.325
Generation 15, Best Cost: 1093329.875
Generation 16, Best Cost: 1087596.9499999997
Generation 17, Best Cost: 1067863.225
Generation 18, Best Cost: 1043737.2250000001
Generation 19, Best Cost: 1043737.2250000001
Generation 20, Best Cost: 1027573.9125000002
Generation 21, Best Cost: 1015784.7375
Generation 22, Best Cost: 1004506.8874999998
Generation 23, Best Cost: 1003593.7124999999
Generation 24, Best Cost: 981425.7749999

In [4]:
# Load the problem data
file_path = "Dataset/wl_50"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1500,  # Adjust as needed
    generations=1500,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 2072201.3499999999
Generation 2, Best Cost: 1995405.3125
Generation 3, Best Cost: 1944031.0125
Generation 4, Best Cost: 1884135.1875
Generation 5, Best Cost: 1752737.8124999998
Generation 6, Best Cost: 1752737.8124999998
Generation 7, Best Cost: 1675174.7375000007
Generation 8, Best Cost: 1629781.7375
Generation 9, Best Cost: 1568530.2749999997
Generation 10, Best Cost: 1525271.7874999999
Generation 11, Best Cost: 1521801.0625
Generation 12, Best Cost: 1479276.2999999998
Generation 13, Best Cost: 1424372.5499999998
Generation 14, Best Cost: 1387032.9625
Generation 15, Best Cost: 1379420.4999999998
Generation 16, Best Cost: 1323421.2125
Generation 17, Best Cost: 1323421.2125
Generation 18, Best Cost: 1301448.7125000001
Generation 19, Best Cost: 1293517.8250000002
Generation 20, Best Cost: 1271218.9124999999
Generation 21, Best Cost: 1244893.5875
Generation 22, Best Cost: 1238680.2875
Generation 23, Best Cost: 1203009.775
Generation 24, Best Cost: 1190603.0124999

In [6]:
# Load the problem data
file_path = "Dataset/wl_200"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1500,  # Adjust as needed
    generations=1500,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 43137.85237000001
Generation 2, Best Cost: 40992.426419999996
Generation 3, Best Cost: 40992.426419999996
Generation 4, Best Cost: 40992.426419999996
Generation 5, Best Cost: 40992.426419999996
Generation 6, Best Cost: 40363.60459
Generation 7, Best Cost: 40363.60459
Generation 8, Best Cost: 40192.105059999994
Generation 9, Best Cost: 39108.32806999999
Generation 10, Best Cost: 39108.32806999999
Generation 11, Best Cost: 38550.28573000001
Generation 12, Best Cost: 38161.11225
Generation 13, Best Cost: 38161.11225
Generation 14, Best Cost: 37739.65992
Generation 15, Best Cost: 37739.65992
Generation 16, Best Cost: 37089.26734
Generation 17, Best Cost: 36324.50505
Generation 18, Best Cost: 36324.50505
Generation 19, Best Cost: 35923.302930000005
Generation 20, Best Cost: 35766.75402
Generation 21, Best Cost: 35349.893260000004
Generation 22, Best Cost: 34200.307349999995
Generation 23, Best Cost: 34200.307349999995
Generation 24, Best Cost: 33979.65886
Generation

In [7]:
# Load the problem data
file_path = "Dataset/wl_300"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1500,  # Adjust as needed
    generations=1500,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 99820.46840000001
Generation 2, Best Cost: 99820.46840000001
Generation 3, Best Cost: 98297.68417000001
Generation 4, Best Cost: 97431.90831999999
Generation 5, Best Cost: 97410.57257
Generation 6, Best Cost: 97216.40319
Generation 7, Best Cost: 95824.45453999999
Generation 8, Best Cost: 95824.45453999999
Generation 9, Best Cost: 94736.75688
Generation 10, Best Cost: 94502.83361999999
Generation 11, Best Cost: 93633.45594000001
Generation 12, Best Cost: 93633.45594000001
Generation 13, Best Cost: 93340.47056
Generation 14, Best Cost: 92883.32643000002
Generation 15, Best Cost: 92087.3291
Generation 16, Best Cost: 92087.3291
Generation 17, Best Cost: 91525.90913
Generation 18, Best Cost: 91507.54453
Generation 19, Best Cost: 91507.54453
Generation 20, Best Cost: 91148.78663000002
Generation 21, Best Cost: 89185.71334
Generation 22, Best Cost: 89185.71334
Generation 23, Best Cost: 87796.58381000001
Generation 24, Best Cost: 87060.32387000001
Generation 25, Best C

In [8]:
# Load the problem data
file_path = "Dataset/wl_500"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)

# Initialize the genetic algorithm
ga = GeneticAlgorithm(
    num_depots=num_depots,
    num_customers=num_customers,
    depot_capacities=depot_capacities,
    setup_costs=setup_costs,
    customer_demands=customer_demands,
    cost_matrix=cost_matrix,
    population_size=1500,  # Adjust as needed
    generations=1500,  # Adjust as needed
    mutation_rate=0.5,  # Adjust as needed
    crossover_rate=1.0 # Adjust as needed
)

# Run the genetic algorithm
best_solution, best_cost = ga.evolve()

# Print the results
print("Best Solution (Depot Assignments):", best_solution)
print("Best Cost:", best_cost)

Generation 1, Best Cost: 97056.15642
Generation 2, Best Cost: 96994.52211
Generation 3, Best Cost: 96994.52211
Generation 4, Best Cost: 96415.52476
Generation 5, Best Cost: 95137.02442999999
Generation 6, Best Cost: 95137.02442999999
Generation 7, Best Cost: 95134.18942
Generation 8, Best Cost: 95134.18942
Generation 9, Best Cost: 94211.33357
Generation 10, Best Cost: 94211.33357
Generation 11, Best Cost: 93799.07383000001
Generation 12, Best Cost: 93233.43873999998
Generation 13, Best Cost: 92401.63122
Generation 14, Best Cost: 92304.52262999999
Generation 15, Best Cost: 91941.83816
Generation 16, Best Cost: 91941.83816
Generation 17, Best Cost: 91200.94797000001
Generation 18, Best Cost: 90458.59735
Generation 19, Best Cost: 89169.74045
Generation 20, Best Cost: 89169.74045
Generation 21, Best Cost: 88510.20078
Generation 22, Best Cost: 88510.20078
Generation 23, Best Cost: 87226.32548
Generation 24, Best Cost: 86729.19345
Generation 25, Best Cost: 86046.47148
Generation 26, Best Cos